In [141]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["fasalDB"]

In [309]:
#prints all items from all 3 collections. Note: Sample items were added manually through robo3T

collections=["customer", "farm", "Device"]

for collectionName in collections:
    for e in mydb[collectionName].find():
      print(e)
    print("\n")

{'_id': '600c1806289947de938c68ea', 'name': 'Ram Mohan', 'age': 32, 'gender': 'male', 'dateJoined': '2019-09-30T09:18:29.044Z', 'state': 'MH', 'country': 'IN'}


{'_id': 'uTMEtfc77M8eLdX7E', 'customer': '600c1806289947de938c68ea', 'name': 'farm_tomato', 'crop': 'tomato', 'deviceId': '7HGRXY', 'lat': 20.632189, 'long': 74.8961}


{'_id': 'uTMEtfc77M8eLdX7E', 'deviceId': '7HGRXY', 'TC': 24, 'HUM': 84, 'datetime': '2021-07-30T09:18:29.044Z', 'LW': 0, 'PLV2': 0}
{'_id': ObjectId('614490d05d926dfeb1973b5c'), 'deviceId': '7HGRXY', 'TC': 34, 'HUM': 89, 'datetime': '2021-07-30T10:25:29.044Z', 'LW': 0.1, 'PLV2': 0.2}
{'_id': ObjectId('614491085d926dfeb1973b87'), 'deviceId': '7HGRXY', 'TC': 26, 'HUM': 81, 'datetime': '2021-07-30T11:30:29.044Z', 'LW': 0, 'PLV2': 0.1}
{'_id': ObjectId('614491085d926dfeb1973b8a'), 'deviceId': '7HGRXY', 'TC': 14, 'HUM': 87, 'datetime': '2021-07-30T11:10:29.044Z', 'LW': 0.1, 'PLV2': 0.1}




## Question 1

This function returns hourly sensor values for a given crop and input date range 

In [414]:
def get_data(crop_name, start_date, end_date, sensor_list):
    
    #queries into 'farm collection' with crop_name and gets the farm data having that crop
    foundFarm=mydb["farm"].find_one({"crop":crop_name})
    
    if(foundFarm==None):
        return "Crop not found"
    
    cropDeviceId=foundFarm["deviceId"]
    
    #used cropDeviceId to query into 'Device collection' and gets all items within input time range
    deviceData=mydb["Device"].find({
        "deviceId":cropDeviceId, 
         "datetime": {
            "$gte": start_date,
            "$lte":end_date
         }
    })
    
    deviceData.sort("datetime",pymongo.ASCENDING)
    
    for e in deviceData:
        print(e["TC"],e["HUM"], e["LW"], e["PLV2"])

In [415]:
get_data(
    "tomato",
    "2021-07-30T09:18:29.044Z",
    "2021-07-31T09:18:29.044Z",
    [])

24 84 0 0
34 89 0.1 0.2
14 87 0.1 0.1
26 81 0 0.1


## Question 2

In [1]:
import pandas as pd
import dateutil

In [2]:
df=pd.read_csv("./Assignment/SampleData.csv")
df.head()

,deviceId,date,month,hour,TC,HUM,LW,PLV2,farmId,crop name
0,ZT1FC3FS,25-01-2021,1,14,27.130,33.030,0.0,0.0,2CkGqcWtBndq5pHCh,Pomegranate
1,ZT1FC3FS,25-01-2021,1,15,26.110,34.665,0.0,0.0,2CkGqcWtBndq5pHCh,Pomegranate
2,ZT1FC3FS,25-01-2021,1,16,23.625,40.940,0.0,0.0,2CkGqcWtBndq5pHCh,Pomegranate
3,ZT1FC3FS,25-01-2021,1,17,18.430,57.315,0.0,0.0,2CkGqcWtBndq5pHCh,Pomegranate
4,ZT1FC3FS,25-01-2021,1,18,14.065,68.265,0.0,0.0,2CkGqcWtBndq5pHCh,Pomegranate


In [40]:
def getfromDF(crop_name, start_date, end_date, sensor_list):
    filteredDf=df[
        (df['crop name']==crop_name) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)]
    return filteredDf[sensor_list]
# getfromDF(
#     "Pomegranate",
#     "26-01-2021", 
#     "27-01-2021",
#     ["TC","HUM","LW","PLV2"])

In [6]:
df['date']=df['date'].apply(dateutil.parser.parse, dayfirst=True)

## Prepare table1

**Step 1: Calculate Avg. Daily Temp (℃)**

In [12]:
avg_temp=df.groupby('date')["TC"].mean()
avg_temp

date
2021-01-25    15.797500
2021-01-26    12.386250
2021-01-27    11.767708
2021-01-28    13.056250
2021-01-29    11.803750
                ...    
2021-05-27    30.907083
2021-05-28    29.547083
2021-05-29    28.276667
2021-05-30    28.107500
2021-05-31    23.566000
Name: TC, Length: 127, dtype: float64

**Step 2: LW >1% (Daily Total Hours)**

In [146]:
#this function takes grouped data of each day as input and returns the value count of hours for when LW>1% 

def calcLwHrs(day):
    hourCount=0
    for eachHour in day:
        if(eachHour>1):
            hourCount+=1
    return hourCount

#Apply the 'calcLwHrs' to each day
lwHrs=df.groupby('date')['LW'].apply(calcLwHrs)

In [147]:
table1=pd.DataFrame({'avg_temp':avg_temp, 'daily_LW_hrs': lwHrs})
table1.head()

,avg_temp,daily_LW_hrs
date,,
2021-01-25,15.797500,0
2021-01-26,12.386250,2
2021-01-27,11.767708,0
2021-01-28,13.056250,0
2021-01-29,11.803750,3


**Step 3: Calculate Risk column based on Conditions**

In [300]:
def calRiskTab1(eachDay):
    avgTemp=eachDay['avg_temp']
    lwPerDay=eachDay['daily_LW_hrs']
    
    if avgTemp<5:
        return 'Low'
    elif avgTemp>=5 and avgTemp<18 and lwPerDay<14:
        return 'Low'
    elif avgTemp>=5 and avgTemp<18 and lwPerDay>=14:
        return 'Medium'
    elif avgTemp>=18 and avgTemp<25 and lwPerDay<6:
        return 'Low'
    elif avgTemp>=18 and avgTemp<25 and lwPerDay>=6 and lwPerDay<12:
        return 'Medium'
    elif avgTemp>=18 and avgTemp<25 and lwPerDay>=12:
        return 'High'
    elif avgTemp>=25 and avgTemp<30 and lwPerDay<10:
        return 'Low'
    elif avgTemp>=25 and avgTemp<30 and lwPerDay>=10:
        return 'Medium'
    elif avgTemp>=30:
        return 'Low'

In [301]:
#Computes and adds 'risk' column
table1['Risk']=table1.apply(calRiskTab1,axis=1)

**table 1 is ready with temperature (T) + leaf wetness (LW) table and Risk column based on Conditions**

In [131]:
table1.head()

,avg_temp,daily_LW_hrs,Risk
date,,,
2021-01-25,15.797500,0,Low
2021-01-26,12.386250,2,Low
2021-01-27,11.767708,0,Low
2021-01-28,13.056250,0,Low
2021-01-29,11.803750,3,Low


## Prepare table2

In [656]:
RainfallPerDay=pd.DataFrame()
RainfallPerDay['rainfall']=df.groupby('date')['PLV2'].sum() #calculating total rainfall in 24-hour

**Step 1: Calculating 10 day avg rainfall**

In [657]:
tenDayAvg={}   #store the results inform of {key,value} = {date,avg_rain}

#this func calculates 10 day avg and stores it in our 'tenDayAvg' dict
def calcTenDayAvg(PLV2):
    avg = 0
    iterCount = 1
    
    for index,values in PLV2.iterrows():
        avg+=values.rainfall
        if(iterCount%10==0 or iterCount==len(PLV2)):
            tenDayAvg[index]=avg/10
            avg=0
        iterCount+=1

calcTenDayAvg(RainfallPerDay)

In [658]:
# Converting the 'tenDayAvg' dictionary to new table2
table2=pd.DataFrame({"Avg_rainfall":tenDayAvg.values()},index=tenDayAvg.keys())
table2.index.name='date'  #our date column is index setting index name to 'date'

**Step 2: Calculating risk column for 10 day avg rainfall**

In [659]:
def calRiskTab2(everyTenDay):
    avgRainfall=everyTenDay['Avg_rainfall']
    
    if (avgRainfall>=0.28 and avgRainfall<=3) or avgRainfall>=25:
        return 'Low'
    elif avgRainfall>=3 and avgRainfall<7:
        return 'Medium'
    elif avgRainfall>=7 and avgRainfall<25:
        return 'High'
    else:
        return float('NaN')

In [660]:
table2['Risk']=table2.apply(calRiskTab2,axis=1)

In [661]:
 #We don't have any 'High' values. We have got nan for cases where avgRainfall=0
table2['Risk'].unique()

array([nan, 'Low', 'Medium'], dtype=object)

In [662]:
#merge table 1 and table 2 along with there risks column
finalDf=pd.merge(table1,table2,how='outer',left_on='date',right_on='date')

### Final table is almost ready it just needs a 'Spray' Column

In [663]:
finalDf[8:12]  #verifying that our 10 day avg_rainfall is displayed at correct row location

,avg_temp,daily_LW_hrs,Risk_x,Avg_rainfall,Risk_y
date,,,,,
2021-02-02,16.718333,0,Low,NaN,NaN
2021-02-03,17.591250,0,Low,0.0,NaN
2021-02-04,18.457500,0,Low,NaN,NaN
2021-02-05,17.185417,0,Low,NaN,NaN


**In the above dataframe 'Risk_x' is calculated risk based on avg_temp and daily_LW_hrs, where as 'Risk_y' is risk calculated based on 10 days Avg_rainfall**

In [666]:
# --> RUN ONCE --> then '#'comment it out

# finalDf['date']=finalDf.index
# finalDf.index=range(1,len(finalDf)+1)

# finalDf.columns.tolist()
# finalDf=finalDf[['date','avg_temp', 'daily_LW_hrs', 'Risk_x', 'Avg_rainfall', 'Risk_y']]

**Earlier we had 'date' as our index but in the above code I have done slight modifications in the format**
  - Converted index column to a normal dataframe column
  - Introduced a serial number indexing
  - re-ordered the columns such that 'date' column is placed first
  
  
 Let's have a look at our modified dataframe

In [667]:
finalDf.fillna("NA",inplace=True)

In [668]:
finalDf.head()

,date,avg_temp,daily_LW_hrs,Risk_x,Avg_rainfall,Risk_y
1,2021-01-25,15.797500,0,Low,NA,NA
2,2021-01-26,12.386250,2,Low,NA,NA
3,2021-01-27,11.767708,0,Low,NA,NA
4,2021-01-28,13.056250,0,Low,NA,NA
5,2021-01-29,11.803750,3,Low,NA,NA


**Let's introduce a Spray column and fill it with only '0's**

Later we will put **'1's on the dates** using calcSprayInterval() func **based on conditions provided**

In [669]:
finalDf['spray']=0

Let's calculate the value of spray column based on conditions provided. 

**Approach:**

**calcSprayInterval() func:**
- We have used a dict **'countRisks'** to store count of each risk

- So, based on risks accumulated in 10 days, if there are **8 Medium risks**(1H=2M) then we will advise for spray on the **11th day** else spray on the **15th day**


**calcMediums() func:** 
 - We are using the formula **'1H=2M'**, so when a 'Medium' occurs we will increment the **cntMediums by 1** and
 - when a High occurs then we will increase **cntMediums by 2**

So the **calcMediums()** functions computes total number of mediums based on medium and **high risks**

In [670]:
def calcSprayInterval(copyDf):
    countRisks={'Low':0, 'Medium':0, 'High':0, 'NA':0} 
    for index,value in copyDf.iterrows():
        
        risk=riskByPrecedence(value['Risk_x'], value['Risk_y'])
        countRisks[risk]+=1
        
        if(index%10==0):
            totMediums=calcMediums(countRisks)
            
            if(totMediums>8):
                #Spray on 11th day
                copyDf['spray'][index+1]=1  
            else: 
                #Spray on 15th day
                copyDf['spray'][index+5]=1

            countRisks={'Low':0, 'Medium':0, 'High':0, 'NA':0} 
            #re-initialise the dict to 0, as we will require it to calculate count of risks for next 10 intervals
    return copyDf

def calcMediums(countRisks):
    totMediums=0
    totMediums+=countRisks['Medium']
    totMediums+=2*countRisks['High']
    
    return totMediums

# this func takes 2 risks and returns the higher risk
def riskByPrecedence(risk_x, risk_y):
    if(risk_x=='High' or risk_y=='High'):
        return 'High'
    elif(risk_x=='Medium' or risk_y=='Medium'):
        return 'Medium'
    elif(risk_x=='Low' or risk_y=='Low'):
        return 'Low'
    else:
        return 'NA'

In [671]:
#We have created a copy dataframe and called the calcSprayInterval() function, 
# Since, it's providing the desired result we are overwrite our finalDf
finalDf=calcSprayInterval(finalDf.copy())

/Users/sudhanshukumar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [672]:
finalDf[:30]

,date,avg_temp,daily_LW_hrs,Risk_x,Avg_rainfall,Risk_y,spray
1,2021-01-25,15.797500,0,Low,NA,NA,0
2,2021-01-26,12.386250,2,Low,NA,NA,0
3,2021-01-27,11.767708,0,Low,NA,NA,0
4,2021-01-28,13.056250,0,Low,NA,NA,0
5,2021-01-29,11.803750,3,Low,NA,NA,0
6,2021-01-30,12.098333,0,Low,NA,NA,0
7,2021-01-31,13.884167,0,Low,NA,NA,0
8,2021-02-01,15.496250,0,Low,NA,NA,0
9,2021-02-02,16.718333,0,Low,NA,NA,0
10,2021-02-03,17.591250,0,Low,0.0,NA,0
